# Iterative algorithms for MDPs, demonstrated via a simple Pac-Man game

The agent moves within a maze and tries to reach an exit field with a positive reward, while not being captured by the ghost in the maze. The agent's locomotion rules agree with *plain_maze*, and the ghost moves randomly by one field per step, with preferred direction towards the agent. The state space consists of all possible agent and ghost locations, plus a final "game over" state.

In [1]:
import numpy as np

In [2]:
# iterative algorithms and utility functions for Markov Decision Processes (MDPs)
import mdp

In [3]:
# environment
from env import MazeGhostEnv

In [4]:
# show description of maze (grid world) geometry
with open('maze_geometry2.txt', 'r') as file:
    for line in file:
        print(line, end='')

# S: start, X: inaccessible, E: exit with reward +1, F: exit with reward -1
..XE
.X..
.X.F
S...


In [5]:
# "reward" on regular fields
r = -0.04

In [6]:
# define environment
e = MazeGhostEnv('maze_geometry2.txt', r)

In [7]:
# discount factor
gamma = 0.99

In [8]:
# perform value iteration
u = mdp.value_iteration(e.tprob, e.rewards, gamma)
print('value function for ghost at first accessible location:')
if hasattr(np, 'printoptions'):
    with np.printoptions(precision=3):
        print(e.maze_array(u[:len(e.locs)]))
else:
    print(e.maze_array(u[:len(e.locs)]))

value iteration with epsilon=1e-14 completed after 154 iterations
value function for ghost at first accessible location:
[[-1.223 -1.242    nan  1.   ]
 [-1.235    nan  0.824  0.924]
 [-1.141    nan  0.563 -1.   ]
 [-2.     0.035  0.302  0.064]]


In [9]:
# optimal policy corresponding to u;
# in some cases the agent tries to "jump over" the ghost
pol = mdp.policy_from_utility(e.tprob, u)
print('optimal policy (all possible ghost locations):')
print(e.draw_policy(pol))

optimal policy (all possible ghost locations):
↓ ← █ E
↑ █ → ↑
↓ █ ↑ F
G → ↑ ←

↓ ← █ E
↓ █ → ↑
↑ █ ↑ F
→ G ↑ ↓

↓ ← █ E
↓ █ → ↑
↓ █ ↑ F
← → G ←

↓ ← █ E
↓ █ → ↑
↓ █ ↑ F
→ ↑ ↑ G

↑ ← █ E
↓ █ → ↑
G █ ↑ F
→ → ↑ ←

↓ ← █ E
↓ █ → ↑
↓ █ G F
→ ↑ ↑ ↓

↓ ← █ E
↓ █ ↑ ↑
↓ █ ↑ G
→ → ↓ ←

↓ → █ E
G █ → ↑
↓ █ ↑ F
→ → ↑ ←

↓ ← █ E
↓ █ G ↑
↓ █ ↓ F
→ → ↓ ←

↓ ← █ E
↓ █ → G
↓ █ ← F
→ → ↑ ↓

G ← █ E
↓ █ → ↑
↓ █ ↑ F
→ → ↑ ←

↓ G █ E
↓ █ → ↑
↓ █ ↑ F
→ → ↑ ←

↓ ← █ G
↓ █ → ↑
↓ █ ↑ F
→ → ↑ ←


In [10]:
# consistency check
if gamma < 1:
    upol = mdp.utility_from_policy(e.tprob, e.rewards, gamma, pol)
    uerr = np.linalg.norm(upol - u)
    print('utility from policy consistency check error:', uerr)

utility from policy consistency check error: 3.046221841855575e-15


In [11]:
# alternative: policy iteration
if gamma < 1:
    pal = mdp.policy_iteration(e.tprob, e.rewards, gamma)
    palerr = np.linalg.norm((pal - pol) * e.pmask)
    print('policy iteration consistency check error:', palerr)

policy iteration completed after 4 iterations
policy iteration consistency check error: 0.0


In [12]:
# Q-value function
Q = mdp.q_iteration(e.tprob, e.rewards, gamma)

Q-value iteration with epsilon=1e-14 completed after 139 iterations


In [13]:
# can obtain utility from Q-value function
uQ = mdp.utility_from_qvalue(Q)
uQerr = np.linalg.norm(u - uQ)
print('utility from Q-value consistency check error:', uQerr)

utility from Q-value consistency check error: 1.1012004949980631e-14


In [14]:
# can obtain policy from Q-value function
pQ = mdp.policy_from_qvalue(Q)
pQerr = np.linalg.norm((pQ - pol) * e.pmask)
print('policy from Q-value consistency check error:', pQerr)

policy from Q-value consistency check error: 0.0


In [15]:
# play a game
e.play(pol, gamma)

step 0
reward: -0.04
G ░ █ E
░ █ ░ ░
░ █ ░ F
☺ ░ ░ ░
action: →
_____________
step 1
reward: -0.04
G ░ █ E
░ █ ░ ░
░ █ ░ F
░ ☺ ░ ░
action: →
_____________
step 2
reward: -0.04
░ ░ █ E
G █ ░ ░
░ █ ░ F
░ ░ ☺ ░
action: ↑
_____________
step 3
reward: -0.04
░ ░ █ E
G █ ░ ░
░ █ ☺ F
░ ░ ░ ░
action: ↑
_____________
step 4
reward: -0.04
░ ░ █ E
░ █ ☺ ░
G █ ░ F
░ ░ ░ ░
action: →
_____________
step 5
reward: -0.04
░ ░ █ E
░ █ ░ ☺
G █ ░ F
░ ░ ░ ░
action: ↑
_____________
step 6
reward: -0.04
░ ░ █ E
░ █ ☺ ░
░ █ ░ F
G ░ ░ ░
action: →
_____________
step 7
reward: -0.04
░ ░ █ E
░ █ ░ ░
G █ ☺ F
░ ░ ░ ░
action: ↑
_____________
step 8
reward: -0.04
░ ░ █ E
G █ ☺ ░
░ █ ░ F
░ ░ ░ ░
action: →
_____________
step 9
reward: -0.04
░ ░ █ E
G █ ░ ☺
░ █ ░ F
░ ░ ░ ░
action: ↑
_____________
step 10
reward: 1.0
░ ░ █ ☺
G █ ░ ░
░ █ ░ F
░ ░ ░ ░
action: →
_____________
Game over!
cumulative discounted reward (gamma = 0.99): 0.5219103750440224
